# Transforming files to parameters.
## Should run after "Contact Patterns" notebook!!

### In order to run this notebook One should make sure the above files exist:
* Data/raw
    * Data/raw/asymptomatic_proportions.csv
    * Data/raw/population_size.csv                    
    * Data/raw/init_i_italy.csv                                
    * Data/raw/IL_initial_eps.csv                     
    * Data/raw/vent_proba.csv 
    * Data/raw/risk_dist.csv
    
* Data/base_contact_mtx
    * Data/base_contact_mtx/full_home_mtx.csv
    * Data/base_contact_mtx/full_leisure_mtx_no_school.csv
    * Data/base_contact_mtx/full_leisure_mtx_no_work.csv
    * Data/base_contact_mtx/full_leisure_mtx_routine.csv
    * Data/base_contact_mtx/full_work_mtx_no_school.csv
    * Data/base_contact_mtx/full_work_mtx_no_work.csv
    * Data/base_contact_mtx/full_work_mtx_routine.csv
    
* Data/demograph
    * Data/demograph/age_dist_area.csv
    * Data/demograph/religion_dis.csv
    * Data/demograph/sick_prop.csv

### This Notebook generates the above files:
* Data/parameters
    - Data/parameters/f0_full.pickle
    - Data/parameters/eps_dict.pickle
    - Data/parameters/eps_by_region.pickle
    - Data/parameters/hospitalization.pickle
    - Data/parameters/C_calibration.pickle
    - Data/parameters/orthodox_dist.pickle
    - Data/parameters/init_pop.pickle

# Imports

In [42]:
import numpy as np
import pandas as pd
from matplotlib.patches import Patch
import itertools
import pickle
from matplotlib import pyplot as plt
import datetime
import scipy
from scipy import optimize
from scipy.sparse import csr_matrix
import sys
import os
sys.path.append('../SEIR_full/')
sys.path.append('..')
from SEIR_full.indices import *
%matplotlib inline

In [43]:
len(N)

720

# Asymptomatic

In [44]:
asymp = pd.read_csv('../Data/raw/asymptomatic_proportions.csv', index_col=0)
asymp.head()

,Scenario 1,Scenario 2,Scenario 3,Unnamed: 4
Age,,,,
0-4,0.957467,0.765973,0.612779,NaN
5-9,0.957467,0.765973,0.612779,NaN
10-19,0.972167,0.777734,0.622187,NaN
20-29,0.921424,0.737139,0.589711,NaN
30-39,0.899804,0.719843,0.575874,NaN


In [45]:
f0_full = {} # dict that contains the possible scenarios

# asymptomatic with risk group, high risk with 0
f_init = np.zeros(len(list(itertools.product(R.values(), A.values()))))
for i in [1,2,3]:
    f_tmp = f_init.copy()
    f_tmp[9:] = asymp['Scenario '+ str(i)].values[:-1]
    f0_full['Scenario'+ str(i)] = expand_partial_array(risk_age_dict, f_tmp)

In [46]:
# Save
try:
    os.mkdir('../Data/parameters')
except:
    pass
with open('../Data/parameters/f0_full.pickle', 'wb') as handle:
    pickle.dump(f0_full, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Initial illness

In [47]:
# Age dist. positive specimens
age_dist = {'0-4':0.02, '5-9':0.02, '10-19':0.11, '20-29':0.23, '30-39':0.15, '40-49':0.14, '50-59':0.14, '60-69':0.11,
           '70+':0.08}

In [48]:
age_dist_area = pd.read_csv('../Data/demograph/age_dist_area.csv')

In [49]:
age_dist_area.drop(['Unnamed: 0'],axis=1, inplace=True)

In [50]:
age_dist_area.set_index('cell_id', inplace=True)

In [51]:
age_dist_area

,0-4,5-9,10-19,20-29,30-39,40-49,50-59,60-69,70+
cell_id,,,,,,,,,
11,0.014308,0.012920,0.022441,0.018661,0.013881,0.010796,0.008755,0.007243,0.007149
11_betshemesh,0.002288,0.002029,0.002915,0.001865,0.001517,0.001078,0.000667,0.000415,0.000322
21,0.001522,0.001433,0.002633,0.002342,0.001849,0.001726,0.001468,0.001217,0.000975
22,0.001473,0.001373,0.002504,0.002217,0.001743,0.001615,0.001398,0.001147,0.000927
23,0.005437,0.005364,0.010731,0.009133,0.007059,0.007134,0.005740,0.004239,0.003480
24,0.006357,0.006477,0.013212,0.011283,0.009157,0.009012,0.007119,0.005036,0.004055
29,0.000388,0.000401,0.000768,0.000656,0.000536,0.000541,0.000426,0.000347,0.000268
31,0.005286,0.004879,0.008517,0.008946,0.008915,0.007956,0.006960,0.007405,0.007854
32,0.004867,0.004823,0.008960,0.007326,0.006645,0.006415,0.005091,0.003883,0.003394


In [52]:
age_dist_area = age_dist_area.stack()

In [53]:
age_dist_area

cell_id       
11       0-4      0.014308
         5-9      0.012920
         10-19    0.022441
         20-29    0.018661
         30-39    0.013881
                    ...   
71       30-39    0.005760
         40-49    0.004169
         50-59    0.002805
         60-69    0.002029
         70+      0.001272
Length: 180, dtype: float64

In [54]:
init_pop = expand_partial_array(region_age_dict, age_dist_area.values)
init_pop.shape

(720,)

In [55]:
init_pop[inter_dict['Intervention']]=0
init_pop

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [56]:
risk_pop = pd.read_csv('../Data/raw/risk_dist.csv')
risk_pop.set_index('Age', inplace=True)
risk_pop['High'] = risk_pop['risk']
risk_pop['Low'] = 1 - risk_pop['risk']
risk_pop.drop(['risk'], axis=1, inplace=True)
risk_pop = risk_pop.stack()
risk_pop.index = risk_pop.index.swaplevel(0, 1)
risk_pop = risk_pop.unstack().stack()
risk_pop

      Age  
High  0-4      0.050
      5-9      0.106
      10-19    0.106
      20-29    0.149
      30-39    0.149
      40-49    0.149
      50-59    0.330
      60-69    0.421
      70+      0.512
Low   0-4      0.950
      5-9      0.894
      10-19    0.894
      20-29    0.851
      30-39    0.851
      40-49    0.851
      50-59    0.670
      60-69    0.579
      70+      0.488
dtype: float64

In [57]:
for (r, a), g_idx in zip(risk_age_dict.keys(), risk_age_dict.values()):
    init_pop[g_idx] = init_pop[g_idx] * risk_pop[r,a]
    
# Age distribution:
pop_dist = init_pop

In [58]:
# Save
with open('../Data/parameters/init_pop.pickle', 'wb') as handle:
    pickle.dump(pop_dist, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [59]:
len(R)*len(A)

18

In [60]:
# risk distribution by age:
risk_dist = pd.read_csv('../Data/raw/population_size.csv')

init_I_dis_italy = pd.read_csv('../Data/raw/init_i_italy.csv')['proportion'].values[:-1]

f_init = pd.read_pickle('../Data/parameters/f0_full.pickle')

eps_t = {}

In [61]:
init_I_dis_italy.sum()

0.9999999999999999

In [62]:
init_I_IL = {}
init_I_dis = {}
for i in [1,2,3]:
    scen = 'Scenario'+str(i)
    f_init_i = f_init[scen][:(len(R)*len(A))]
    init_I_IL[scen] = (491./(1-(f_init_i * risk_dist['pop size'].values).sum())) / 9136000.
    init_I_dis[scen] = init_I_dis_italy * init_I_IL[scen]

In [63]:
for i in [1,2,3]:
    scen = 'Scenario'+str(i)
    eps_t[scen] = []
    for i in range(1000):
        eps_t[scen].append(init_I_dis[scen][i] * pop_dist if i < len(init_I_dis[scen]) else 0)

In [64]:
# Save
with open('../Data/parameters/eps_dict.pickle', 'wb') as handle:
    pickle.dump(eps_t, handle, protocol=pickle.HIGHEST_PROTOCOL)

## eps by region proportion

In [65]:
# Loading data
region_prop = pd.read_csv('../Data/demograph/sick_prop.csv', index_col=0)['cases_prop'].copy()
region_prop.index = region_prop.index.astype(str)
risk_prop = pd.read_csv('../Data/raw/risk_dist.csv', index_col=0)['risk'].copy()
print(region_prop.head())
print()
print(risk_prop.head())

cell_id
11    0.135317
21    0.009418
22    0.017790
23    0.033524
24    0.018210
Name: cases_prop, dtype: float64

Age
0-4      0.050
5-9      0.106
10-19    0.106
20-29    0.149
30-39    0.149
Name: risk, dtype: float64


In [66]:
eps_t_region = {}

In [67]:
for sc, init_I in zip(init_I_dis.keys(), init_I_dis.values()):
    eps_temp = []
    for t in range(1000):
        if t < len(init_I):
            # empty array for day t
            day_vec = np.zeros(len(N))
            # fill in the array, zero for intervention groups
            for key in N.keys():
                if N[key][0] == 'Intervention':
                    day_vec[key] = 0
                else:
                    day_vec[key] = init_I[t] * region_prop[N[key][1]] * age_dist[N[key][3]] * \
                    (risk_prop[N[key][3]]**(1 - (N[key][2] == 'Low'))) * \
                    ((1 - risk_prop[N[key][3]])**(N[key][2] == 'Low'))
            eps_temp.append(day_vec)
        else:
             eps_temp.append(0.0)
        
        eps_t_region[sc] = eps_temp


In [68]:
# save eps:
with open('../Data/parameters/eps_by_region.pickle', 'wb') as handle:
    pickle.dump(eps_t_region, handle, protocol=pickle.HIGHEST_PROTOCOL)  

## Smoothed eps by region -------------- UNFINISHED -------------------

In [134]:
########## UNFINISHED ############
init_eps = pd.read_csv('../Data/raw/IL_initial_eps.csv', header=None).values[:,0].copy()
init_eps

array([  0.78180071,   2.94105921,   5.84488904,   8.5253491 ,
        12.02483762,  17.497441  ,  24.38473153,  33.09622265,
        41.99385742,  39.83459892,  75.79742365,  93.14595037,
       114.9991361 , 143.6279275 , 172.5917768 , 219.0158361 ,
       274.5609032 , 341.4606921 ])

In [135]:
init_eps_asymp = {}
for i in [1,2,3]:
    scen = 'Scenario'+str(i)
    f_init_i = f_init[scen][:(len(R)*len(A))]
    init_eps_asymp[scen] = (init_eps/(1-(f_init_i * risk_dist['pop size'].values).sum())) / 9136000.

In [136]:
f_init['Scenario3'][:(len(R)*len(A))]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.6127786 ,
       0.6127786 , 0.62218712, 0.58971125, 0.57587446, 0.59021178,
       0.56415614, 0.54046208, 0.14933333])

In [137]:
init_eps_asymp['Scenario3'].sum()*9136000

3003.15972721764

In [138]:
def fill_eps_day(init_eps, dic=mdl.N, age_)

SyntaxError: invalid syntax (<ipython-input-138-958ad179e108>, line 1)

In [ ]:
eps_reg = []


# Hospitalizations

In [69]:
hosp_init = pd.read_csv('../Data/raw/vent_proba.csv')
hosp_init

,Unnamed: 0,num_positives,mortality,vent_pr
0,0-4,250.02,0.000000,0.000000
1,5-9,250.02,0.000000,0.000000
2,10-19,1375.11,0.000000,0.000000
3,20-29,2875.23,2.817460,0.000980
4,30-39,1875.15,2.817460,0.001503
5,40-49,1750.14,5.634921,0.003220
6,50-59,1750.14,10.000000,0.005714
7,60-69,1375.11,30.992063,0.022538
8,70+,1000.08,216.944444,0.216927


In [70]:
hosp = expand_partial_array(age_dict, hosp_init['vent_pr'].values)

In [71]:
hosp

array([0.        , 0.        , 0.        , 0.00097991, 0.00150253,
       0.0032197 , 0.00571383, 0.02253788, 0.21692709, 0.        ,
       0.        , 0.        , 0.00097991, 0.00150253, 0.0032197 ,
       0.00571383, 0.02253788, 0.21692709, 0.        , 0.        ,
       0.        , 0.00097991, 0.00150253, 0.0032197 , 0.00571383,
       0.02253788, 0.21692709, 0.        , 0.        , 0.        ,
       0.00097991, 0.00150253, 0.0032197 , 0.00571383, 0.02253788,
       0.21692709, 0.        , 0.        , 0.        , 0.00097991,
       0.00150253, 0.0032197 , 0.00571383, 0.02253788, 0.21692709,
       0.        , 0.        , 0.        , 0.00097991, 0.00150253,
       0.0032197 , 0.00571383, 0.02253788, 0.21692709, 0.        ,
       0.        , 0.        , 0.00097991, 0.00150253, 0.0032197 ,
       0.00571383, 0.02253788, 0.21692709, 0.        , 0.        ,
       0.        , 0.00097991, 0.00150253, 0.0032197 , 0.00571383,
       0.02253788, 0.21692709, 0.        , 0.        , 0.     

In [72]:
# Save
with open('../Data/parameters/hospitalization.pickle', 'wb') as handle:
    pickle.dump(hosp, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Calibration contact matrix

Important : here no intervention mean the course of interventions made by government

In [73]:
full_mtx_home = scipy.sparse.load_npz('../Data/base_contact_mtx/full_home.npz')

full_mtx_work = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_work_no_bb.npz'),
}

full_mtx_leisure = {
    'routine': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_routine.npz'),
    'no_school': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_school.npz'),
    'no_work': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_work.npz'),
    'no_100_meters': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_100_meters.npz'),
    'no_bb': scipy.sparse.load_npz('../Data/base_contact_mtx/full_leisure_no_bb.npz'),
}

In [74]:
C_calibration = {}
d_tot = 500

In [75]:
# no intervation are null groups
home_no_inter = []
work_no_inter = []
leis_no_inter = []

for i in range(d_tot):
    home_no_inter.append(csr_matrix((full_mtx_home.shape[0], full_mtx_home.shape[1])))
    work_no_inter.append(csr_matrix((full_mtx_work['routine'].shape[0], full_mtx_work['routine'].shape[1])))
    leis_no_inter.append(csr_matrix((full_mtx_leisure['routine'].shape[0], full_mtx_leisure['routine'].shape[1])))

In [76]:
# Intervantion
home_inter = []
work_inter = []
leis_inter = []

# first days of routine from Feb 21st - March 13th
d_rout = 9+13
for i in range(d_rout):
    home_inter.append(full_mtx_home)
    work_inter.append(full_mtx_work['routine'])
    leis_inter.append(full_mtx_leisure['routine'])

# first days of no school from March 14th - March 16th
d_no_school = 3
for i in range(d_no_school):
    home_inter.append(full_mtx_home)
    work_inter.append(full_mtx_work['no_school'])
    leis_inter.append(full_mtx_leisure['no_school'])

# without school and work from March 17th - March 25th
d_no_work = 9
for i in range(d_no_work):
    home_inter.append(full_mtx_home)
    work_inter.append(full_mtx_work['no_work'])
    leis_inter.append(full_mtx_leisure['no_work'])

# 100 meters constrain from March 26th - April 2nd
d_no_100_meters = 8
for i in range(d_no_100_meters):
    home_inter.append(full_mtx_home)
    work_inter.append(full_mtx_work['no_100_meters'])
    leis_inter.append(full_mtx_leisure['no_100_meters'])
    
# Bnei Brak quaranrine from April 3rd
for i in range(d_tot-d_no_school-d_rout-d_no_work-d_no_100_meters):
    home_inter.append(full_mtx_home)
    work_inter.append(full_mtx_work['no_bb'])
    leis_inter.append(full_mtx_leisure['no_bb'])

In [77]:
C_calibration['home_inter'] = home_no_inter
C_calibration['work_inter'] = work_no_inter
C_calibration['leisure_inter'] = leis_no_inter
C_calibration['home_non'] = home_inter
C_calibration['work_non'] = work_inter
C_calibration['leisure_non'] = leis_inter

In [78]:
# Save
with open('../Data/parameters/C_calibration.pickle', 'wb') as handle:
    pickle.dump(C_calibration, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Haredi vector

In [79]:
# Loading raw data
hared_dis = pd.read_csv('../Data/demograph/religion_dis.csv', index_col=0)[['cell_id','Orthodox']].copy()
hared_dis.set_index('cell_id', inplace=True)
hared_dis.head()

,Orthodox
cell_id,
11,0.183911
11_betshemesh,0.484846
21,0.129815
22,0.057856
23,0.008882


In [80]:
# Creating model orthodox dist. and save it as pickle
model_orthodox_dis = np.zeros(len(GA))
for i in GA.keys():
    model_orthodox_dis[i] = hared_dis.loc[str(GA[i][0])]
    
with open('../Data/parameters/orthodox_dist.pickle', 'wb') as handle:
    pickle.dump(model_orthodox_dis, handle, protocol=pickle.HIGHEST_PROTOCOL)  

In [81]:
model_orthodox_dis

array([1.83911164e-01, 1.83911164e-01, 1.83911164e-01, 1.83911164e-01,
       1.83911164e-01, 1.83911164e-01, 1.83911164e-01, 1.83911164e-01,
       1.83911164e-01, 4.84846467e-01, 4.84846467e-01, 4.84846467e-01,
       4.84846467e-01, 4.84846467e-01, 4.84846467e-01, 4.84846467e-01,
       4.84846467e-01, 4.84846467e-01, 1.29814749e-01, 1.29814749e-01,
       1.29814749e-01, 1.29814749e-01, 1.29814749e-01, 1.29814749e-01,
       1.29814749e-01, 1.29814749e-01, 1.29814749e-01, 5.78560383e-02,
       5.78560383e-02, 5.78560383e-02, 5.78560383e-02, 5.78560383e-02,
       5.78560383e-02, 5.78560383e-02, 5.78560383e-02, 5.78560383e-02,
       8.88231633e-03, 8.88231633e-03, 8.88231633e-03, 8.88231633e-03,
       8.88231633e-03, 8.88231633e-03, 8.88231633e-03, 8.88231633e-03,
       8.88231633e-03, 1.41964797e-03, 1.41964797e-03, 1.41964797e-03,
       1.41964797e-03, 1.41964797e-03, 1.41964797e-03, 1.41964797e-03,
       1.41964797e-03, 1.41964797e-03, 3.41781274e-04, 3.41781274e-04,
      